# Übung Datenbanken: Ein Zugauskunfssystem
![](https://www.klipfel-mathys.ch/images/imagesCAUM394L.jpg)

In [2]:
# Hier ist nur Code zum Initialisieren der Umgebeung, bitte gehen Sie weiter, es gibt nichts zu sehen.

# Keine langen Fehlermeldungen
import sys
ipython = get_ipython()

def exception_handler(exception_type, exception, traceback):
    print("%s: %s" % (exception_type.__name__, exception), file=sys.stderr)

ipython._showtraceback = exception_handler

# Lade die Erweiterung, damit wir SQL Befehle nutzen können
%reload_ext sql
%sql sqlite:///zugauskunft-kemper.db
%sql PRAGMA foreign_keys = ON

 * sqlite:///zugauskunft-kemper.db
Done.


[]

# Aufgabe

Nachdem wir in der letzten Aufgabe unsere Daten in folgendes Datenmodell importiert haben:

![](https://nextcloud.th-deg.de/s/Kc7m486Z3KbZyB4/download?path=%2F&files=bahn-er.png)

Wollen wir in dieser Aufgabe einige Abfragen auf diesem Modell entwerfen.

In [10]:
# !echo ".schema" | sqlite3 zugauskunft-kemper.db
# CREATE TABLE Zug (
#   ZugNr INTEGER NOT NULL,
#   Name TEXT NOT NULL,
#   StartBhf TEXT NOT NULL,
#   EndBhf TEXT NOT NULL,
#   FOREIGN KEY(StartBhf) REFERENCES Bahnhof(BahnhofId),
#   FOREIGN KEY(EndBhf) REFERENCES Bahnhof(BahnhofId),
#   PRIMARY KEY(ZugNr)
# );

# CREATE TABLE Bahnhof (
#   BahnhofId TEXT NOT NULL,
#   Name TEXT NOT NULL,
#   PRIMARY KEY(BahnhofId)
# );

# CREATE TABLE verbindet (
#   ZugNr Integer NOT NULL,
#   VonBahnhof TEXT NOT NULL,
#   NachBahnhof TEXT,
#   Abfahrt TIME NOT NULL,
#   Ankunft TIME NOT NULL,
#   FOREIGN KEY(VonBahnhof) REFERENCES Bahnhof(BahnhofId),
#   FOREIGN KEY(NachBahnhof) REFERENCES Bahnhof(BahnhofId),
#   FOREIGN KEY(ZugNr) REFERENCES Zug(ZugNr),
#   PRIMARY KEY(VonBahnhof, ZugNr)
# );

In [8]:
%%sql

SELECT * FROM Zug LIMIT 5;

 * sqlite:///zugauskunft-kemper.db
Done.


ZugNr,Name,StartBhf,EndBhf
1003122052,WBA1 Plattling,de:09271:5500,de:09271:4445
1003122056,WBA1 Plattling,de:09271:5500,de:09271:4445
1003122057,WBA1 Plattling,de:09271:5500,de:09271:4445
1003122058,WBA1 Plattling,de:09276:5025_G,de:09271:4445
1003122059,WBA1 Plattling,de:09276:4726,de:09271:4445


In [6]:
%%sql

SELECT * FROM verbindet LIMIT 5;


 * sqlite:///zugauskunft-kemper.db
Done.


ZugNr,VonBahnhof,NachBahnhof,Abfahrt,Ankunft
1003122052,de:09271:5500,de:09271:80502,5:30:00,5:30:00
1003122052,de:09271:80502,de:09271:4445,5:35:00,5:35:00
1003122052,de:09271:4445,None,5:39:00,5:39:00
1003122056,de:09271:5500,de:09271:80502,24:38:00,24:38:00
1003122056,de:09271:80502,de:09271:4445,24:44:00,24:45:00


### Abfrage 1: Abfahrten in Deggendorf

Entwerfen Sie eine Abfrage, die alle Abfahrten in Deggendorf anzeigt. Geben Sie die Zugnummer, den Namen, den Start- und Endbahnhof sowie die Abfahrtszeit aus.

In [11]:
%sql SELECT ZugNr,MIN(Abfahrt) as Abfahrt FROM verbindet GROUP BY ZugNr LIMIT 10;

 * sqlite:///zugauskunft-kemper.db
Done.


ZugNr,Abfahrt
1003122052,5:30:00
1003122056,24:38:00
1003122057,23:29:00
1003122058,22:04:00
1003122059,21:41:00
1003122061,18:16:00
1003122062,17:16:00
1003122063,13:16:00
1003122064,12:16:00
1003122065,19:59:00


In [31]:
%%sql

WITH min_abfahrt AS (
    SELECT ZugNr, MIN(Abfahrt) as Abfahrt
    FROM verbindet
    GROUP BY ZugNr
)
SELECT Zug.ZugNr, Zug.Name, start.Name as 'Startbahnhof', end.Name as 'Endbahnhof', verbindet.Abfahrt
FROM Zug
JOIN Bahnhof AS start ON(Zug.StartBhf = start.BahnhofId)
JOIN Bahnhof AS end ON(Zug.EndBhf = end.BahnhofId)
JOIN verbindet USING(ZugNr)
JOIN min_abfahrt ON verbindet.ZugNr = min_abfahrt.ZugNr AND verbindet.Abfahrt = min_abfahrt.Abfahrt
WHERE start.Name LIKE 'Deggendorf%'
ORDER BY verbindet.Abfahrt ASC
LIMIT 10;


 * sqlite:///zugauskunft-kemper.db
Done.


ZugNr,Name,Startbahnhof,Endbahnhof,Abfahrt
1015320361,WBA1 Plattling,Deggendorf Hbf,Plattling,10:45:00
1015320360,WBA1 Plattling,Deggendorf Hbf,Plattling,11:45:00
1003122064,WBA1 Plattling,Deggendorf Hbf,Plattling,12:16:00
1015320364,WBA1 Plattling,Deggendorf Hbf,Plattling,12:45:00
1003122063,WBA1 Plattling,Deggendorf Hbf,Plattling,13:16:00
1015320359,WBA1 Plattling,Deggendorf Hbf,Plattling,13:45:00
1015320371,WBA1 Plattling,Deggendorf Hbf,Plattling,14:45:00
1015320357,WBA1 Plattling,Deggendorf Hbf,Plattling,15:45:00
1003122087,WBA1 Plattling,Deggendorf Hbf,Plattling,16:16:00
1015320356,WBA1 Plattling,Deggendorf Hbf,Plattling,16:45:00


In [17]:
%%sql
SELECT Zug.ZugNr, Zug.Name, verbindet.Abfahrt, start.Name as Startbahnhof, end.Name as Endbahnhof
FROM Zug
JOIN Bahnhof AS start
ON(Zug.StartBhf = start.BahnhofId)
JOIN Bahnhof AS end
ON(Zug.EndBhf = end.BahnhofId)
JOIN (SELECT ZugNr,MIN(Abfahrt) as Abfahrt FROM verbindet GROUP BY ZugNr)
AS verbindet
USING(ZugNr)
WHERE Startbahnhof LIKE 'Deggendorf%'
ORDER BY Abfahrt ASC
LIMIT 10;



 * sqlite:///zugauskunft-kemper.db
Done.


ZugNr,Name,Abfahrt,Startbahnhof,Endbahnhof
1015320361,WBA1 Plattling,10:45:00,Deggendorf Hbf,Plattling
1015320360,WBA1 Plattling,11:45:00,Deggendorf Hbf,Plattling
1003122064,WBA1 Plattling,12:16:00,Deggendorf Hbf,Plattling
1015320364,WBA1 Plattling,12:45:00,Deggendorf Hbf,Plattling
1003122063,WBA1 Plattling,13:16:00,Deggendorf Hbf,Plattling
1015320359,WBA1 Plattling,13:45:00,Deggendorf Hbf,Plattling
1015320371,WBA1 Plattling,14:45:00,Deggendorf Hbf,Plattling
1015320357,WBA1 Plattling,15:45:00,Deggendorf Hbf,Plattling
1003122087,WBA1 Plattling,16:16:00,Deggendorf Hbf,Plattling
1015320356,WBA1 Plattling,16:45:00,Deggendorf Hbf,Plattling


Sie finden in den Ergebnissen sicher eine Zeilen mit : Abfahrt 25:27. Das ist typisch für Bahn Systeme, kann aber manche Zeitfunktion Ihrer Datenbank aus dem Tritt bringen.

## Aufgabe 2: Sortierng

In [36]:
%%sql

SELECT * FROM verbindet LIMIT 5;

 * sqlite:///zugauskunft-kemper.db
Done.


ZugNr,VonBahnhof,NachBahnhof,Abfahrt,Ankunft
1003122052,de:09271:5500,de:09271:80502,05:30:00,05:30:00
1003122052,de:09271:80502,de:09271:4445,05:35:00,05:35:00
1003122052,de:09271:4445,None,05:39:00,05:39:00
1003122056,de:09271:5500,de:09271:80502,24:38:00,24:38:00
1003122056,de:09271:80502,de:09271:4445,24:44:00,24:45:00


In [35]:
%%sql

UPDATE verbindet SET Abfahrt = "0" || Abfahrt WHERE Abfahrt LIKE "_:%";

 * sqlite:///zugauskunft-kemper.db
0 rows affected.


[]

In [34]:
%%sql

UPDATE verbindet SET Ankunft = "0" || Ankunft WHERE Ankunft LIKE "_:%";

 * sqlite:///zugauskunft-kemper.db
484 rows affected.


[]

## Abfrage 3: Laufweg einer zufälligen Fahrt

Geben Sie alle Halte einer zufälligen (Hint: Subselect mit RANDOM()) Fahrt aus.

In [43]:
%%sql

SELECT Zug.ZugNr, Zug.Name, verbindet.Ankunft, verbindet.Abfahrt, start.Name as VonBahnhof, end.Name as NachBahnhof
FROM Zug, verbindet, Bahnhof AS start, Bahnhof AS end
WHERE
verbindet.ZugNr = Zug.ZugNr AND
verbindet.VonBahnhof = start.BahnhofId AND
verbindet.NachBahnhof = end.BahnhofId AND
Zug.ZugNr = (SELECT ZugNr FROM Zug ORDER BY RANDOM() LIMIT 3);

 * sqlite:///zugauskunft-kemper.db
Done.


ZugNr,Name,Ankunft,Abfahrt,VonBahnhof,NachBahnhof
1003122250,WBA2 Zwiesel (Bay),13:29:00,13:29:00,Bodenmais,Böhmhof
1003122250,WBA2 Zwiesel (Bay),13:32:00,13:32:00,Böhmhof,Langdorf
1003122250,WBA2 Zwiesel (Bay),13:40:00,13:40:00,Langdorf,Außenried
1003122250,WBA2 Zwiesel (Bay),13:43:00,13:43:00,Außenried,Zwiesel (Bay)


### Abfrage 3: Von Deggendorf nach Grafenau

Wie komme ich von Deggendorf nach Grafenau? Gibt es einen einfachen Weg das in einer SQL Abfrage abzubilden?